In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
# print(X.shape)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [5]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.NewtonRaphson(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.1, line_search_method='backtrack', line_search_cond='armijo')
# opt = torch_numopt.NewtonRaphson(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='wolfe')
# opt = torch_numopt.NewtonRaphson(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='strong-wolfe')
# opt = torch_numopt.NewtonRaphson(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='goldstein')

all_loss = {}
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss[epoch+1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch+1] += loss
    all_loss[epoch+1] /= len(data_loader)
    print(', loss: {}'.format(all_loss[epoch+1].detach().numpy().item()))

epoch:  0, loss: 0.48321083188056946
epoch:  1, loss: 0.3647404611110687
epoch:  2, loss: 0.3077298104763031
epoch:  3, loss: 0.277543842792511
epoch:  4, loss: 0.2310977578163147
epoch:  5, loss: 0.1910809576511383
epoch:  6, loss: 0.15777739882469177
epoch:  7, loss: 0.12976552546024323
epoch:  8, loss: 0.11073324084281921
epoch:  9, loss: 0.0975651666522026
epoch:  10, loss: 0.08475934714078903
epoch:  11, loss: 0.05099410563707352
epoch:  12, loss: 0.029096025973558426
epoch:  13, loss: 0.024074960500001907
epoch:  14, loss: 0.02120845951139927
epoch:  15, loss: 0.019706856459379196
epoch:  16, loss: 0.019423071295022964
epoch:  17, loss: 0.01864047907292843
epoch:  18, loss: 0.01643044874072075
epoch:  19, loss: 0.015252364799380302
epoch:  20, loss: 0.015168093144893646
epoch:  21, loss: 0.01443713903427124
epoch:  22, loss: 0.01352209597826004
epoch:  23, loss: 0.013054951094090939
epoch:  24, loss: 0.012964457273483276
epoch:  25, loss: 0.012851009145379066
epoch:  26, loss: 0.